# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-04-01 23:11:48.523752: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:11:48.523778: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.3.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

2022-04-01 23:11:49.844141: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-01 23:11:50.206171: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-01 23:11:50.206194: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jschneiderm-A70-MOB): /proc/driver/nvidia/version does not exist
2022-04-01 23:11:50.206628: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-01 23:11:50.231223: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2304000000 Hz
2022-04-01 23:11:50.23

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='elu'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 352us/step - loss: 0.5712 - accuracy: 0.7154
Epoch 2/100
250/250 [==============================] - 0s 349us/step - loss: 0.4501 - accuracy: 0.8018
Epoch 3/100
250/250 [==============================] - 0s 357us/step - loss: 0.4294 - accuracy: 0.8077
Epoch 4/100
250/250 [==============================] - 0s 349us/step - loss: 0.4229 - accuracy: 0.8129
Epoch 5/100
250/250 [==============================] - 0s 358us/step - loss: 0.4158 - accuracy: 0.8170
Epoch 6/100
250/250 [==============================] - 0s 359us/step - loss: 0.4071 - accuracy: 0.8229
Epoch 7/100
250/250 [==============================] - 0s 358us/step - loss: 0.3968 - accuracy: 0.8273
Epoch 8/100
250/250 [==============================] - 0s 349us/step - loss: 0.3865 - accuracy: 0.8315
Epoch 9/100
250/250 [==============================] - 0s 336us/step - loss: 0.3781 - accuracy: 0.8381
Epoch 10/100
250/250 [==============================] - 0s 336us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [15]:
y_pred = ann.predict(X_test)
y_pred

array([[0.24794465],
       [0.24555126],
       [0.12901613],
       ...,
       [0.13877565],
       [0.18571529],
       [0.25018293]], dtype=float32)

In [16]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1501   94]
 [ 186  219]]


---
# Avaliando o Modelo

In [18]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [20]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

2022-04-01 23:11:59.876638: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:11:59.876668: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-01 23:11:59.906498: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:11:59.906523: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-01 23:11:59.907352: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or dire

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 0s 418us/step - loss: 0.5023 - accuracy: 0.7956
Epoch 2/10
720/720 [==============================] - 0s 423us/step - loss: 0.4854 - accuracy: 0.7968
Epoch 2/10
720/720 [==============================] - 0s 425us/step - loss: 0.4865 - accuracy: 0.7962
Epoch 2/10
720/720 [==============================] - 0s 436us/step - loss: 0.5046 - accuracy: 0.7939
Epoch 2/10
 73/720 [==>...........................] - ETA: 0s - loss: 0.4234 - accuracy: 0.8055

2022-04-01 23:12:02.483286: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-01 23:12:02.483319: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jschneiderm-A70-MOB): /proc/driver/nvidia/version does not exist
2022-04-01 23:12:02.483596: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-01 23:12:02.492231: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2304000000 Hz
2022-04-01 23:12:02.494299: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dc9ae59910 initialized for platform Host (this does not guarantee tha

144/720 [=====>........................] - ETA: 0s - loss: 0.4305 - accuracy: 0.8042Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 0s 671us/step - loss: 0.4292 - accuracy: 0.7962
Epoch 3/10
720/720 [==============================] - 0s 666us/step - loss: 0.4297 - accuracy: 0.7969
Epoch 3/10
720/720 [==============================] - 0s 655us/step - loss: 0.4297 - accuracy: 0.7967
Epoch 3/10
720/720 [==============================] - 0s 664us/step - loss: 0.4325 - accuracy: 0.7944
Epoch 3/10
720/720 [==============================] - 1s 793us/step - loss: 0.4246 - accuracy: 0.7969=================>...........] - ETA: 0s - loss: 0.4971 - accuracy: 0.7931 - ETA: 0s - loss: 0.5143 - accuracy: 0.79
 - 1s 798us/step - loss: 0.4240 - accuracy: 0.7962
Epoch 4/10
720/720 [==============================]  1/720 [..............................] - 1s 793us/step - loss: 0.4262 - accuracy: 0.7967
 - ETA: 0s - loss: 0.3020 - accuracy: 0.9000Epoch 4/10
720/720 [===========

In [21]:
print(mean,variance)

0.8341249942779541 0.010063078792592522


# Melhorando o modelo

In [22]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10


2022-04-01 23:12:09.391304: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:12:09.391343: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-01 23:12:09.391528: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:12:09.391558: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-01 23:12:09.408351: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or dire

720/720 [==============================] - 0s 682us/step - loss: 0.4911 - accuracy: 0.7961: 0s - loss: 0.4991 - accuracy: 0.7982 - ETA: 0s - loss: 0.5946 - accuracy: 0.80: 0s - loss: 0.6812 - accuracy: 0.7694 - ETA: 0s - loss: 0.6769 - accuracy: 
 - 0s 682us/step - loss: 0.4980 - accuracy: 0.7954
Epoch 2/10Epoch 2/10

720/720 [==============================] - 0s 691us/step - loss: 0.4918 - accuracy: 0.7953
Epoch 2/10
720/720 [==============================] - 1s 726us/step - loss: 0.4861 - accuracy: 0.7967
Epoch 2/10
720/720 [==============================] - 1s 735us/step - loss: 0.4379 - accuracy: 0.7961 - loss: 0.4356 - accuracy: 0.7990 - ETA: 0s - loss: 0.4366 - accuracy: 0.79
Epoch 3/10
720/720 [==============================] - 1s 757us/step - loss: 0.4373 - accuracy: 0.7962
Epoch 3/10
720/720 [==============================] - 1s 785us/step - loss: 0.4306 - accuracy: 0.7957
Epoch 3/10
720/720 [==============================] - 1s 770us/step - loss: 0.4349 - accuracy: 0.7969
Epo

2022-04-01 23:12:10.887723: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-01 23:12:10.890712: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-01 23:12:10.909881: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-01 23:12:10.966900: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1


720/720 [==============================] - 1s 722us/step - loss: 0.4325 - accuracy: 0.7961
Epoch 4/10
720/720 [==============================] - 0s 670us/step - loss: 0.4322 - accuracy: 0.7969
Epoch 4/10
720/720 [==============================] - 1s 698us/step - loss: 0.4288 - accuracy: 0.7957
Epoch 4/10
720/720 [==============================] - 1s 739us/step - loss: 0.4329 - accuracy: 0.7962
Epoch 4/10
113/720 [===>..........................] - ETA: 0s - loss: 0.4332 - accuracy: 0.8088

2022-04-01 23:12:11.118139: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-01 23:12:11.129427: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1


720/720 [==============================] - 0s 485us/step - loss: 0.4306 - accuracy: 0.7961
Epoch 5/10
720/720 [==============================] - 0s 482us/step - loss: 0.4292 - accuracy: 0.7969
Epoch 5/10
720/720 [==============================] - 0s 482us/step - loss: 0.4246 - accuracy: 0.7957
Epoch 5/10
720/720 [==============================] - 0s 485us/step - loss: 0.4332 - accuracy: 0.7962
Epoch 5/10
720/720 [==============================] - 0s 428us/step - loss: 0.4275 - accuracy: 0.7961
Epoch 6/10
720/720 [==============================] - 0s 444us/step - loss: 0.4286 - accuracy: 0.7969
Epoch 6/10
720/720 [==============================] - 0s 444us/step - loss: 0.4178 - accuracy: 0.7957
Epoch 6/10
720/720 [==============================] - 0s 447us/step - loss: 0.4293 - accuracy: 0.7962
Epoch 6/10
427/720 [================>.............] - ETA: 0s - loss: 0.4242 - accuracy: 0.8133

2022-04-01 23:12:11.930535: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-01 23:12:11.930659: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jschneiderm-A70-MOB): /proc/driver/nvidia/version does not exist
2022-04-01 23:12:11.930975: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-01 23:12:11.932715: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-01 23:12:11.932722: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_N

494/720 [===================>..........] - ETA: 0s - loss: 0.4294 - accuracy: 0.8117Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 0s 671us/step - loss: 0.4287 - accuracy: 0.8157
Epoch 7/10
720/720 [==============================] - 0s 683us/step - loss: 0.4259 - accuracy: 0.8090
Epoch 7/10
720/720 [==============================] - 1s 727us/step - loss: 0.4270 - accuracy: 0.7986
Epoch 7/10
720/720 [==============================] - 1s 795us/step - loss: 0.4141 - accuracy: 0.8161
Epoch 7/10
720/720 [==============================] - 1s 827us/step - loss: 0.4308 - accuracy: 0.8229: 0s - loss: 0.6040 - accuracy: 0.79 ETA: 0s - loss: 0.5967 - accuracy: 0.80
Epoch 8/10
720/720 [==============================] - 1s 851us/step - loss: 0.4245 - accuracy: 0.8217
Epoch 8/10
720/720 [==============================] - 1s 862us/step - loss: 0.4087 - accuracy: 0.8235
Epoch 8/10
720/720 [==============================] - 1s 859us/step - l

In [24]:
print(mean,variance)

0.8200000107288361 0.017526759584037658


# Tuning the ANN

In [25]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [26]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 0s 326us/step - loss: 0.5092 - accuracy: 0.7965
Epoch 2/50
720/720 [==============================] - 0s 333us/step - loss: 0.4432 - accuracy: 0.7971
Epoch 3/50
720/720 [==============================] - 0s 357us/step - loss: 0.4375 - accuracy: 0.7971
Epoch 4/50
720/720 [==============================] - 0s 333us/step - loss: 0.4326 - accuracy: 0.7971
Epoch 5/50
720/720 [==============================] - 0s 331us/step - loss: 0.4304 - accuracy: 0.7971
Epoch 6/50
720/720 [==============================] - 0s 311us/step - loss: 0.4296 - accuracy: 0.7971
Epoch 7/50
720/720 [==============================] - 0s 325us/step - loss: 0.4290 - accuracy: 0.7971
Epoch 8/50
720/720 [==============================] - 0s 338us/step - loss: 0.4262 - accuracy: 0.8057
Epoch 9/50
720/720 [==============================] - 0s 318us/step - loss: 0.4274 - accuracy: 0.8214
Epoch 10/50
720/720 [==============================] - 0s 301us/step - loss: 0.424

In [27]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8385


# Halving Grid Search

In [28]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [29]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

2022-04-01 23:34:20.474408: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:34:20.474443: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-01 23:34:20.480041: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:34:20.480053: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-01 23:34:20.480067: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your mac

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2022-04-01 23:34:25.883756: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-01 23:34:25.884756: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jschneiderm-A70-MOB): /proc/driver/nvidia/version does not exist
2022-04-01 23:34:25.885624: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-01 23:34:25.894121: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-01 23:34:25.894206: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear 

80/80 [==============================] - 0s 2ms/step - loss: 0.6763 - accuracy: 0.8210
Epoch 2/50
Epoch 1/50
 1/80 [..............................] - ETA: 0s - loss: 0.6931 - accuracy: 0.4000Epoch 1/50
Epoch 1/50
Epoch 1/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6789 - accuracy: 0.7835TA: 0s - loss: 0.6873 - accuracy: 0.7792 - ETA: 0s - loss: 0.6880 - accuracy: 0.80
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.5967 - accuracy: 0.8223
Epoch 3/
80/80 [==============================] - ETA: 0s - loss: 0.6415 - accuracy: 0.8000 - 0s 2ms/step - loss: 0.6790 - accuracy: 0.7910
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6761 - accuracy: 0.79220 [==========================>...] - ETA: 0s - loss: 0.6764 - accuracy: 0.80
Epoch 2/
80/80 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.7872
Epoch 3/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6036 - accuracy: 0.7985
E

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


...

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation)

Epoch 1/100
80/80 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.7835
Epoch 49/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4190 - accuracy: 0.8385
Epoch 50/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.7835
Epoch 50/50
 1/80 [..............................] - ETA: 0s - loss: 0.2883 - accuracy: 0.9000

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


80/80 [==============================] - 0s 1ms/step - loss: 0.4260 - accuracy: 0.7835
Epoch 1/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6804 - accuracy: 0.8148
Epoch 2/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6788 - accuracy: 0.7722
Epoch 2/50
80/80 [==============================] - 0s 1ms/step - loss: 0.6403 - accuracy: 0.8223
Epoch 3/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6807 - accuracy: 0.7910
Epoch 2/100
80/80 [==============================] - 0s 957us/step - loss: 0.6805 - accuracy: 0.7960: 0s - loss: 0.6738 - accuracy: 0.70
Epoch 2/100
80/80 [==============================] - 0s 2ms/step - loss: 0.5455 - accuracy: 0.8223
Epoch 4/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6682 - accuracy: 0.8160TA: 0s - loss: 0.6506 - accuracy: 0.80
Epoch 2/50
80/80 [==============================]Epoch 2/100
 - 0s 2ms/step - loss: 0.6733 - accuracy: 0.7810
Epoch 2/50.........................

25/25 [==============================] - 2s 71ms/step - loss: 0.6859 - accuracy: 0.7872
Epoch 54/100
Epoch 60/100
25/25 [==============================] - 2s 72ms/step - loss: 0.6842 - accuracy: 0.7885
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 0.4157 - accuracy: 0.7885
Epoch 41/100
25/25 [==============================] - 0s 1ms/step - loss: 0.4166 - accuracy: 0.8298
 - 0s 1ms/step - loss: 0.6762 - accuracy: 0.7972
25/25 [==============================] - 0s 1ms/step - loss: 0.4216 - accuracy: 0.7985
Epoch 32/100
Epoch 42/100
Epoch 42/100
 1/25 [>.............................] - ETA: 0s - loss: 0.3847 - accuracy: 0.7500Epoch 3/100
 - 0s 1ms/step - loss: 0.6673 - accuracy: 0.8223 - loss: 0.4921 - accuracy: 0.78
25/25 [==============================] - 0s 729us/step - loss: 0.6729 - accuracy: 0.7947
Epoch 3/100
Epoch 3/100
Epoch 43/100.........................] - ETA: 0s - loss: 0.6649 - accuracy: 0.8438
25/25 [==============================] 1/25 [>.......

In [30]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8350438047559449


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)